In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Dataset Preprocessing

Working on dataset obtained from https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data?select=train.csv.zip

This is a labelled dataset.

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv(r'/content/gdrive/MyDrive/train.csv')

In [ ]:
df['total'] = df['insult'] + df['threat'] + df['toxic'] + df['severe_toxic'] + df['obscene'] + df['identity_hate'] 

In [ ]:
df['total']

0         0
1         0
2         0
3         0
4         0
         ..
159566    0
159567    0
159568    0
159569    0
159570    0
Name: total, Length: 159571, dtype: int64

In [ ]:
# original dataset is highly biased
df['total'].value_counts()

0    143346
1      6360
3      4209
2      3480
4      1760
5       385
6        31
Name: total, dtype: int64

### Limiting the imbalance (all column labels = 0) to 25000 entries instead of 143346 for better results.

In [ ]:
count = 0
for i in range(len(df)):
  if df['total'][i] == 0 and count <118346:
    df = df.drop(axis = 0, index = i)
    count = count + 1
  if i % 1000 == 0:
    print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [ ]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,total
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,4
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0,1
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0,1
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1,4
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0,3
...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,0


In [ ]:
df['total'].value_counts()

0    25000
1     6360
3     4209
2     3480
4     1760
5      385
6       31
Name: total, dtype: int64

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.to_csv('/content/gdrive/MyDrive/balanced_train.csv', index = False)

# Find embeddings

In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

     |████████████████████████████████| 79 kB 3.3 MB/s 
     |████████████████████████████████| 3.8 MB 26.1 MB/s 
     |████████████████████████████████| 1.2 MB 42.4 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 895 kB 50.6 MB/s 
     |████████████████████████████████| 6.5 MB 47.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=6ccfd179003034d109e4bd712f73d2665a57cb548107d3e40cf724004727417d
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
lst_embeddings = []

In [ ]:
import pandas as pd
df = pd.read_csv(r'/content/gdrive/MyDrive/balanced_train.csv')

In [ ]:
for idx in range(len(df['comment_text'])):
  if idx%3000==0:
    print(idx)
  sentences_embeddings = sbert_model.encode(df['comment_text'][idx])
  lst_embeddings.append(sentences_embeddings)

0
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000


In [ ]:
df['vectors'] = lst_embeddings

In [ ]:
df.to_csv('/content/gdrive/MyDrive/balanced_train_bert_encoding.csv')

In [ ]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,total,vectors
0,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,4,"[0.7341663, 1.2271725, 1.5535065, -0.27768287,..."
1,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0,1,"[-0.26918796, 0.5250471, 0.7365578, 0.1434066,..."
2,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0,1,"[0.45723635, 0.659503, 2.558805, 0.106181934, ..."
3,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1,4,"[0.053298127, 1.6223435, -0.69390786, -0.29036..."
4,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0,3,"[0.085916676, 1.3194124, 1.1487067, 0.09797474..."
...,...,...,...,...,...,...,...,...,...,...
41220,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,0,"[0.21728995, 0.624204, 0.79717994, -0.00153893..."
41221,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,0,"[-0.1814776, 1.1373698, 0.62726456, 0.02438928..."
41222,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,0,"[0.43388477, 1.0630258, 0.21976353, -0.0082708..."
41223,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,0,"[-0.265276, 0.1898801, 0.6571604, 0.32380867, ..."


In [ ]:
import pandas as pd
import numpy as np
def converter(instr):
    return np.fromstring(instr[1:-1],sep=' ')
df1=pd.read_csv('/content/gdrive/MyDrive/balanced_train_bert_encoding.csv',converters={'vectors':converter})


In [ ]:
type(df1['vectors'][1])

numpy.ndarray

## **Logistic Regression**

For target labels - 'toxic' and 'identity_hate'

In [ ]:
from sklearn.linear_model import LogisticRegression
model_identity_hate_lr = None
model_toxic_lr = None

import copy
def baseline_ml(target):
  X = df1['vectors']
  #print(type(X[1]))
  y = df1[target]
  #print(y)
  print("Logistic Regression for -",target)
  lst = []
  for i in X:
    lst.append(i)
  #print(type(lst[2]))

  from sklearn.model_selection import train_test_split
  x_train, x_test, y_train, y_test = train_test_split(lst, list(y), test_size=0.25, random_state=0)

  model = LogisticRegression(max_iter = 10000)
  model.fit(x_train, y_train)
  #Prediction using Logistic Reg
  if target == 'toxic':
    model_toxic_lr = copy.deepcopy(model)
    
  elif target == 'identity_hate':
    model_identity_hate_lr = copy.deepcopy(model)

  predictions = model.predict(x_test)
  
  #To improve recall
  # proba = model.predict_proba(x_test)
  # if target == 'toxic':
  #   model_toxic_lr = copy.deepcopy(model)
  # elif target == 'identity_hate':
  #   model_identity_hate_lr = copy.deepcopy(model)
  # predictions = []
  # for i in proba:
  #   if i[0] > 0.7:
  #     predictions.append(1)
  #   else:
  #     predictions.append(0)


  import matplotlib.pyplot as plt
  import seaborn as sns
  from sklearn import metrics
  cm = metrics.confusion_matrix(y_test, predictions)
  print("Confusion Mtarix:")
  print(cm)
  precision = metrics.precision_score(y_test, predictions)
  recall = metrics.recall_score(y_test, predictions)
  F1 = 2 * (precision * recall) / (precision + recall)

  print("Accuracy of {} -- {}".format(target, metrics.accuracy_score(y_test, predictions)))
  print("Precision:",precision)
  print("Recall:",recall)
  print("F1 Score - ",F1)

  print("===============")
  print()

  return model

In [ ]:
model_toxic_lr = baseline_ml('toxic')
model_identity_hate_lr = baseline_ml('identity_hate')

Logistic Regression for - toxic
Confusion Mtarix:
[[5977  471]
 [ 572 3287]]
Accuracy of toxic -- 0.8988066362666149
Precision: 0.8746673762639702
Recall: 0.8517750712619849
F1 Score -  0.8630694499146645

Logistic Regression for - identity_hate
Confusion Mtarix:
[[9872   85]
 [ 214  136]]
Accuracy of identity_hate -- 0.9709905889201513
Precision: 0.6153846153846154
Recall: 0.38857142857142857
F1 Score -  0.4763572679509632



## Predicting on unlabelled and unseen dataset

Tagged.csv - Data scraped from wikipedia(unbiased) and google(biased)

In [ ]:
df_tagged = pd.read_csv('/content/gdrive/MyDrive/tagged.csv')

In [ ]:
df_tagged
df_temp = pd.DataFrame()
l_s = []
l_pred = []
for i in range(len(df_tagged['Sentences'])):
  embs = sbert_model.encode(df_tagged['Sentences'][i])
  prediction = model_toxic_lr.predict(embs.reshape(1,-1))  
  l_s.append(df_tagged['Sentences'][i])
  l_pred.append(prediction[0])


In [ ]:
df_temp['sent'] = l_s
df_temp['preds'] = l_pred

In [ ]:
df_temp

,sent,preds
0,'Real Housewives of Miami' reboot includes fra...,0
1,Reserve Bank Puts Off Plunging Country Into In...,0
2,A Latina scientist co-created a new Covid vacc...,0
3,Ayala: Democratic Latino voters may be agonizi...,0
4,India Defense Ministry denies clearance to fil...,0
...,...,...
3613,The only way you're going to get Junior to vis...,0
3614,Sen. Kelly has apparently decided that the sen...,0
3615,What would you expect from this book?,0
3616,"""And what this involved was a steady campaign ...",0


In [ ]:
df_temp.to_csv('/content/gdrive/MyDrive/predicted_bert_toxicity.csv')

### Filtering out sentences that are predicted toxic in nature that is preds = 1.

In [ ]:
list_sentences = []
for i in range(len(df_temp['preds'])):
  if df_temp['preds'][i] == 1:
    list_sentences.append(df_temp['sent'][i])

In [ ]:
list_biased = ['gay','lesbian','black','transgender','chinese']

offensive_word_emb = sbert_model.encode(list_biased)

In [ ]:
lst_embeddings = []
for idx in range(len(df_temp['sent'])):
  if idx%1000==0:
    print(idx)
  sentences_embeddings = sbert_model.encode(df_temp['sent'][idx])
  lst_embeddings.append(sentences_embeddings)

0
1000
2000
3000


In [ ]:
len(lst_embeddings)

3618

In [ ]:
df_temp['vectors'] = lst_embeddings

In [ ]:
# performing custom tagging on test data sentences whose toxicity label was predicted as 1 by the logisitc regression model

list_sentences = []
list_vectors = []
for i in range(len(df_temp['preds'])):
  if df_temp['preds'][i] == 1:
    list_sentences.append(df_temp['sent'][i])
    list_vectors.append(df_temp['vectors'][i])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

new_df = pd.DataFrame()
new_df['sentences'] = list_sentences
new_df['vectors'] = list_vectors
for i in list_biased:
  new_df[i] = ''
for each_vector in range(len(df_temp['vectors'])):
  for each_Word in range(len(offensive_word_emb)):
    cosine = cosine_similarity(offensive_word_emb[each_Word].reshape(1,-1), df_temp['vectors'][each_vector].reshape(1,-1))

    if cosine > 0.3:
      new_df[list_biased[each_Word]][each_vector] = 1
    else:
      new_df[list_biased[each_Word]][each_vector] = 0

In [ ]:
new_df

,sentences,vectors,gay,lesbian,black,transgender,chinese
0,"Damn, he was a tank when he played.","[0.48846555, 0.10022835, 0.4329467, 0.55410224...",0,1,0,0,0
1,Gay People Do Not Have License to Weaponize Ho...,"[0.74382323, 1.1609215, 0.34734997, -0.0813653...",0,0,0,0,0
2,See if you're still a loud and proud dipper be...,"[0.19629015, 0.39600614, 1.5100981, 0.04816987...",0,0,0,0,0
3,Worst Opinion of the Week: Lying Is Bad (Unles...,"[-0.30910966, 0.46161208, 0.59425455, 0.019304...",0,1,0,0,0
4,Newsflash\nGod didn't create people to be same...,"[1.0756998, 0.7635576, 0.2387313, 0.2945373, -...",1,0,0,0,0
...,...,...,...,...,...,...,...
605,Man who threatened to kill Pelosi and Bowser p...,"[0.030994726, 0.32879686, 0.49401057, 0.310633...",1,1,1,1,1
606,Bride-To-Be Branded 'Homophobic' After Telling...,"[-0.95716757, 0.78275394, 0.30296406, 0.287040...",0,1,0,1,0
607,Male lions roar\n\nthe rest spend most of thei...,"[0.005638221, 1.3016384, -0.25012562, -0.19220...",1,1,0,0,0
608,It can hide anywhere to stay warm.,"[-0.2573324, -0.33429858, 1.3973311, 0.0909071...",1,1,1,1,1


In [ ]:
new_df.to_csv("/content/gdrive/MyDrive/bert_custom_tagging_logistic.csv", index = False)